<a href="https://colab.research.google.com/github/rsabilio/ia024-projeto-rag/blob/main/4_rag_recursive_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instruções

* Estratégia
    * Usar segmentação por similaridade
    * Busca densa
    * Recursive Retriver

# Instalando pacotes

In [1]:
!pip install -q llama-index llama-index-llms-groq sentence-transformers llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

# Importando pacotes

In [2]:
import os
from google.colab import userdata

os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [10]:
from llama_index.core import Settings, VectorStoreIndex, PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.core.schema import TextNode, IndexNode
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.evaluation import FaithfulnessEvaluator, CorrectnessEvaluator, RelevancyEvaluator
from llama_index.core.response.notebook_utils import display_source_node, display_response
import json
import pandas as pd
from rich.pretty import pprint
from tqdm import tqdm
from IPython.display import Markdown, display

import nest_asyncio
nest_asyncio.apply()


# GitHub

In [4]:
GITHUB_TOKEN      = userdata.get('GITHUB_TOKEN')
GITHUB_USER_EMAIL = userdata.get('GITHUB_USER_EMAIL')
GITHUB_USER_NAME  = userdata.get('GITHUB_USER_NAME')

!git config --global user.email '''{GITHUB_USER_EMAIL}'''
!git config --global user.name '''{GITHUB_USER_NAME}'''

# Clonar o repositório GitHub
!git clone https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag.git

Cloning into 'ia024-projeto-rag'...
remote: Enumerating objects: 6051, done.
remote: Counting objects: 100% (1056/1056), done.
remote: Compressing objects: 100% (378/378), done.
remote: Total 6051 (delta 665), reused 1045 (delta 658), pack-reused 4995
Receiving objects: 100% (6051/6051), 1.12 GiB | 31.05 MiB/s, done.
Resolving deltas: 100% (2054/2054), done.
Updating files: 100% (5788/5788), done.


In [5]:
DATA_DIR = '/content/ia024-projeto-rag'

In [6]:
def save_on_git(msg):

    %cd /content/ia024-projeto-rag

    !git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

    !git pull origin main

    !git add .

    !git commit -m '''{msg}'''

    !git push origin main

In [7]:
def update_local_repo():
    %cd /content/ia024-projeto-rag

    !git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

    !git pull origin main

In [ ]:
update_local_repo()

# RAG - Recursive Retriever

## Preparando dados

In [8]:
embed_model = HuggingFaceEmbedding(model_name="alfaneo/bertimbau-base-portuguese-sts")

Settings.embed_model = embed_model
Settings.llm         = Groq(model="llama3-70b-8192")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.39k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/438k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [86]:
import glob

def get_json_files(directory):
    return glob.glob(os.path.join(directory, "*.json"))

base_nodes     = []
directory_path = f"{DATA_DIR}/2-dataset/"
json_files     = get_json_files(directory_path)

for json_file in json_files:
    print(f"Carregando: {json_file}")
    with open(json_file, 'r') as f:
        documento = json.load(f)

    documento_titulo = documento['titulo']
    documento_texto  = documento['texto_compacto']

    for parte in documento['partes']:
        metadata = {}
        metadata["documento"] = documento_titulo

        titulo_nome    = parte['titulo']
        titulo_artigos = parte['artigos']
        capitulos      = parte['capitulos']

        metadata["titulo"] = titulo_nome
        if titulo_artigos != "":
            node = TextNode(text=titulo_artigos, metadata=metadata)
            base_nodes.append(node)

        for capitulo in capitulos:
            cap_titulo  = capitulo['titulo']
            cap_artigos = capitulo['artigos']

            metadata["capitulo"] = cap_titulo
            node = TextNode(text=cap_artigos, metadata=metadata)
            base_nodes.append(node)
    print(f"{len(base_nodes)} nodes carregados")

for i, node in enumerate(base_nodes):
    node.id_ = f"node_{i}"

print(f"Total de nodes carregados: {len(base_nodes)}")

Carregando: /content/ia024-projeto-rag/2-dataset/estatuto.json
17 nodes carregados
Carregando: /content/ia024-projeto-rag/2-dataset/organização_didática_da_educação_básica.json
47 nodes carregados
Carregando: /content/ia024-projeto-rag/2-dataset/organização_didática_dos_cursos_de_graduação.json
76 nodes carregados
Total de nodes carregados: 76


In [98]:
pprint(base_nodes[-5:])

[
│   TextNode(
│   │   id_='node_71',
│   │   embedding=None,
│   │   metadata={
│   │   │   'doc_name': 'Organização Didática dos Cursos de Graduação',
│   │   │   'titulo': 'TÍTULO VI DA ORGANIZAÇÃO CURRICULAR E ACADÊMICA',
│   │   │   'capitulo': 'CAPÍTULO II DO CALENDÁRIO ACADÊMICO.'
│   │   },
│   │   excluded_embed_metadata_keys=[],
│   │   excluded_llm_metadata_keys=[],
│   │   relationships={},
│   │   text='Art 171 - O Calendário Acadêmico, independentemente do ano civil, terá, no mínimo, 200 (duzentos) dias letivos de efetivo trabalho escolar/acadêmico, nos turnos matutino, vespertino e noturno, excluindo o tempo reservado aos exames finais, quando houver. Art 172 - Entende-se por dia letivo aquele fixado no calendário acadêmico em que se realizam atividades educacionais (ensino, pesquisa e extensão), dentro ou fora dos câmpus, com a participação conjunta de professores e estudantes. § 1º. O sábado poderá ser considerado dia letivo, desde que previsto em calendário. § 2º. Em todos os níveis e modalidades de ensino ofertados pelo IFSP, deve-se assegurar o cumprimento dos dias letivos e/ou carga horária, considerando suas especificidades. Art 173 - A PRE elaborará um modelo do Calendário Acadêmico do ano letivo subsequente e apresentará a todos os câmpus no mês de setembro, para que seja adaptado por estes, com a colaboração e deliberação dos setores de ensino dos mesmos, e, deverá ser aprovado pelo CONCAM. Art 174 - O Calendário Acadêmico organizará as vivências administrativo- pedagógicas e ordenará a distribuição dos dias letivos previstos por Lei para cada ano, devendo conter: I - previsão de feriados, recessos e períodos destinados à realização de eventos educacionais e culturais; II - dias destinados aos Conselhos de Classe Deliberativo e Pedagógico (para os cursos da educação básica), Colegiado de Curso (para os cursos superiores); III - dias destinados a encontros pedagógicos, incluindo docentes e Técnicos Administrativos, com vistas ao estudo e análise da dinâmica do câmpus e a fim de planejar suas ações; IV - dias destinados a reuniões com pais; V - datas de início e término: a) dos semestres letivos; b) de Planejamento Acadêmico-Pedagógico; c) da entrega do Plano de Ensino; d) de recepção dos estudantes e comunidade; e) de matrícula (inicial, optativas, eletivas, regime especial de dependência, estudante especial, etc); f) de renovação de matrícula (rematrícula); g) de solicitação de transferência, mediante edital; h) de solicitação de reopção de curso, mediante edital; i) de solicitação de aproveitamento de estudos e extraordinário aproveitamento de estudos; j) de solicitação de trancamento de matrícula; k) de reposição de aulas; l) de registro de notas; m) de divulgação das notas para o corpo discente; n) de pedidos de retificação de notas e faltas; o) de férias acadêmicas; p) de recesso administrativo; q) de divulgação dos componentes curriculares optativos e eletivos; r) de reavaliações ou exames finais; s) de cancelamento de disciplinas. Art 175 - Os cursos e programas a distância terão calendário próprio, que deverá: I - respeitar o calendário acadêmico do câmpus; II - conter as atividades presenciais do curso; III - ser publicado e divulgado desde o início do curso. Art 176. Os calendários acadêmicos deverão ser analisados e aprovados pelo Conselho de Câmpus (CONCAM) ou equivalente. Na ausência deste, pelo Diretor-geral, para posterior publicação e envio para a Pró-Reitoria de Ensino (PRE). Parágrafo Único. O calendário acadêmico deverá constar, obrigatoriamente, no Sistema Acadêmico, no site do câmpus e em local visível e de fácil acesso ao público. Art 177 - Cabe ao Coordenador de Curso zelar pelo cumprimento da carga horária, bem como pela forma e procedimentos de reposição de aulas, quando necessário para o cumprimento do Calendário Acadêmico. Art 178 - Em casos fortuitos, o Calendário Acadêmico poderá ser alterado, por ato do Diretor-Geral de cada câmpus, que informará à PRE. Art 179 - O horário

## Construindo Retriever e Query Engine

In [ ]:
# Set Up Recursive Retriever
sub_chunk_sizes = [128, 256, 512]
sub_node_parsers = [SentenceSplitter(chunk_size=c, chunk_overlap=20) for c in sub_chunk_sizes]

all_nodes = []
for base_node in base_nodes:

    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes]
        all_nodes.extend(sub_inodes)

    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)

all_nodes_dict = {n.node_id: n for n in all_nodes}

vector_index = VectorStoreIndex(all_nodes)
vector_index.storage_context.persist(f"{DATA_DIR}/4-vector_index")

print(f"Total de nodes: {len(all_nodes)}")

In [88]:
print(f"Total de nodes: {len(all_nodes)}")

Total de nodes: 2621


In [92]:
vector_retriever = vector_index.as_retriever(similarity_top_k=20)

retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    node_dict=all_nodes_dict,
    verbose=True,
)

query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever,
    verbose=True
)

In [93]:
# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k} " f"**Text:** "
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown(""))

In [94]:
display_prompt_dict(query_engine.get_prompts())

**Prompt Key**: response_synthesizer:text_qa_template **Text:** 

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


**Prompt Key**: response_synthesizer:refine_template **Text:** 

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


In [95]:
new_summary_tmpl_str = ("""
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Provide the source of the answer, such as the document, title, chapter, or article when available.
Answer in the same language as the query.
Query: {query_str}
Source:
Answer:
"""
)

new_refine_tmpl_str = ("""
The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Do not add other information, such as 'The refined answer remains the same,' besides the answer and the source.
Provide the source of the answer, such as the document, title, chapter, or article when available.
Answer in the same language as the query.

Refined Answer:

"""
)

new_summary_tmpl = PromptTemplate(new_summary_tmpl_str)
new_refine_tmpl_str = PromptTemplate(new_refine_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": new_summary_tmpl
     , "response_synthesizer:refine_template" : new_refine_tmpl_str}
)

display_prompt_dict(query_engine.get_prompts())


**Prompt Key**: response_synthesizer:text_qa_template **Text:** 


Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Provide the source of the answer, such as the document, title, chapter, or article when available.
Answer in the same language as the query.
Query: {query_str}
Source:
Answer:



**Prompt Key**: response_synthesizer:refine_template **Text:** 


The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Do not add other information, such as 'The refined answer remains the same,' besides the answer and the source.
Provide the source of the answer, such as the document, title, chapter, or article when available.
Answer in the same language as the query.

Refined Answer: 




In [104]:
#query = "Qual o nome do IFSP?"
#query = "Quais são os campi do IFSP? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência."
#query = "Quantos campi o IFSP tem? Informe onde você encontrou essa informação"
#query = "O IFSP tem só ensino médio? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência."
#query = "Quais artigos do Estatuto dizem que o IFSP oferece mestrado e doutorado?"
#query = "O que diz o Art 6º do estatuto do IFSP?"
#query = "O Art 4º do estatuto do IFSP é sobre o que?"

#query = "Sou aluno de graduação e gostaria de saber o que são as transferências especial e ex oficio."
query = "O que é Estudante Especial?"

response = query_engine.query(query)
display_response(response)

Retrieving with query id None: O que é Estudante Especial?
Retrieved node with id, entering: node_57
Retrieving with query id node_57: O que é Estudante Especial?
Retrieved node with id, entering: node_40
Retrieving with query id node_40: O que é Estudante Especial?
Retrieved node with id, entering: node_75
Retrieving with query id node_75: O que é Estudante Especial?
Retrieved node with id, entering: node_24
Retrieving with query id node_24: O que é Estudante Especial?
Retrieved node with id, entering: node_28
Retrieving with query id node_28: O que é Estudante Especial?
Retrieved node with id, entering: node_25
Retrieving with query id node_25: O que é Estudante Especial?
Retrieved node with id, entering: node_27
Retrieving with query id node_27: O que é Estudante Especial?
Retrieved node with id, entering: node_49
Retrieving with query id node_49: O que é Estudante Especial?
Retrieved node with id, entering: node_26
Retrieving with query id node_26: O que é Estudante Especial?


**`Final Response:`** O estudante especial é aquele que ingressa no IFSP para cursar componentes curriculares isolados, sem vínculo com um curso específico, visando ao cumprimento de componentes curriculares em dependência no curso de origem.

Source: Organização Didática dos Cursos de Graduação, TÍTULO IV DA GESTÃO ACADÊMICA, CAPÍTULO II DA MATRÍCULA, SEÇÃO VIII – DO ESTUDANTE ESPECIAL, Art. 194.

In [106]:
pprint(base_nodes[26])

TextNode(
│   id_='node_26',
│   embedding=None,
│   metadata={
│   │   'doc_name': 'Organização Didática da Educação Básica',
│   │   'titulo': 'TÍTULO III DOS CURSOS DE EDUCAÇÃO BÁSICA PROFISSIONALIZANTE',
│   │   'capitulo': 'CAPÍTULO III DOS CURSOS TÉCNICOS DE NÍVEL MÉDIO NA FORMA INTEGRADA AO ENSINO MÉDIO.'
│   },
│   excluded_embed_metadata_keys=[],
│   excluded_llm_metadata_keys=[],
│   relationships={},
│   text='Art 60 - Serão estruturados em períodos letivos anuais ou semestrais, em turno único ou período integral, com duração de três ou quatro anos, carga horária de acordo com a legislação vigente e distribuição dos dias letivos de acordo com o artigo 17 desta Organização Didática. Parágrafo único. Parte da carga horária poderá ser desenvolvida a distância, quando previstas as atividades não presenciais no Projeto Pedagógico do Curso, respeitando os dispositivos legais vigentes. Art 61 - Os Currículos dos cursos serão organizados em núcleos politécnicos, conforme a seguinte organização: I - Núcleo estruturante comum: relativo às áreas de conhecimento que compõem a formação geral no Ensino Médio, contemplando conteúdos de base científica e cultural basilares para a formação humana integral; II - Núcleo estruturante articulador: relativo às áreas do conhecimento do Ensino Médio e da Educação Profissional, traduzidas em conteúdo de estreita articulação com o curso, por eixo tecnológico, representando elementos expressivos para a integração curricular; III - Núcleo estruturante tecnológico: relativo a conhecimentos da habilitação profissional, de acordo com o campo de conhecimentos do eixo tecnológico, e as regulamentações do exercício da profissão; IV - Prática Profissional: relativo a estágio curricular supervisionado, em situação real de trabalho, optativo ou obrigatório e Projeto Integrador. § 1º. A carga horária total obrigatória dos cursos deverá ser distribuída nos três núcleos estruturantes, de forma que a carga horária referente ao núcleo estruturante articulador poderá compor a carga horária do núcleo estruturante comum ou tecnológico. § 2º. Os componentes curriculares poderão ser organizados em módulos, áreas, projetos, séries anuais, períodos semestrais, ciclos, alternância regular de períodos de estudos e grupos não seriados. Art 62 - As estruturas curriculares deverão seguir as orientações dispostas em capítulo próprio desta Organização Didática. Art 63 - Após a conclusão do curso, inclusive do estágio curricular supervisionado, quando obrigatório, o estudante receberá o diploma de técnico de nível médio. SEÇÃO I - DA VERIFICAÇÃO DO RENDIMENTO ESCOLAR NOS CURSOS TÉCNICOS DE NÍVEL MÉDIO NA FORMA INTEGRADA AO ENSINO MÉDIO Art 64 - Considera-se APROVADO: I - o estudante que obtenha em cada área do conhecimento (Linguagens, Matemática, Ciências da Natureza, Ciências Humanas, Disciplinas Técnicas e Projeto Integrador) média das notas finais igual ou superior a 6,0 (seis), e frequência global mínima de 75% (setenta e cinco por cento) das aulas dadas e demais atividades no período letivo; II - o estudante com frequência global mínima de 75% (setenta e cinco por cento) das aulas dadas e demais atividades no período letivo que, após reavaliação obtenha, média das notas finais igual ou superior a 6,0 (seis) em cada área do conhecimento; III - o estudante com frequência global mínima de 75% (setenta e cinco por cento) das aulas dadas e demais atividades no período letivo, que, após análise do Conselho de Classe Deliberativo, seja considerado aprovado. Art 65 - Considera-se REPROVADO: I - o estudante que obtiver frequência global menor que 75% (setenta e cinco por cento), das aulas dadas e demais atividades no período letivo, independentemente da nota que tiver alcançado; II - o estudante com frequência global mínima de 75% (setenta e cinco por cento) das aulas dadas e demais atividades no período letivo, que, após reavaliação, obtenha média das notas finais menor que 6,0 (seis) em pelo menos uma área do conhecimento 

In [105]:
for node in response.source_nodes:
    display_source_node(node)

**Node ID:** node_57<br>**Similarity:** 0.7486204587012969<br>**Text:** SEÇÃO I - DA MATRÍCULA INICIAL Art 52 - Matrícula é o ato formal pelo qual se dá a vinculação aca...<br>

**Node ID:** node_40<br>**Similarity:** 0.7411532412682735<br>**Text:** SEÇÃO I - DA MATRÍCULA INICIAL Art 173 - Matrícula é o ato formal pelo qual se dá a vinculação ac...<br>

**Node ID:** node_75<br>**Similarity:** 0.7389481987170496<br>**Text:** Art 200 - Esta Organização Didática poderá ser reformulada, quando se fizer necessário, mediante ...<br>

**Node ID:** node_24<br>**Similarity:** 0.7310308866255935<br>**Text:** Art 32 - A Educação Básica no IFSP é desenvolvida por meio da oferta de cursos da educação profis...<br>

**Node ID:** node_28<br>**Similarity:** 0.7280256047931942<br>**Text:** Art 80 - Os cursos de Formação Inicial ou Continuada na Modalidade EJA serão ofertados com objeti...<br>

**Node ID:** node_25<br>**Similarity:** 0.7206679783752297<br>**Text:** Art 50 - O currículo dos cursos será organizado em períodos letivos e constituído por componentes...<br>

**Node ID:** node_27<br>**Similarity:** 0.7203949555841106<br>**Text:** Art 69 - Os cursos técnicos de nível médio na forma integrada ao Ensino Médio, na Modalidade de E...<br>

**Node ID:** node_49<br>**Similarity:** 0.7203103285175413<br>**Text:** Art 6 - O Ensino Superior de Graduação, no Instituto Federal de São Paulo, é desenvolvido pela of...<br>

**Node ID:** node_26<br>**Similarity:** 0.7127995480956059<br>**Text:** Art 60 - Serão estruturados em períodos letivos anuais ou semestrais, em turno único ou período i...<br>

# Avaliando conjunto de perguntas e respostas

In [67]:
pairs = [{
            "question": "Qual o nome do IFSP?",
            "answer": "Instituto Federal de Educação, Ciência e Tecnologia de São Paulo.  Referência: Estatuto, Título I, Capítulo I, Art. 1º"
         }
         ,{
            "question": "Quais são os campi do IFSP?",
            "answer": """II. Campus São Paulo, situado na Rua Pedro Vicente, 625, Canindé, São Paulo.
III. Campus Cubatão, situado na Rua Maria Cristina, 50, Jardim Casqueiro, Cubatão.
IV. Campus Sertãozinho, situado na Rua Américo Ambrósio, 269, Jardim Canaã,
Sertãozinho.
V. Campus Guarulhos, situado na Avenida Salgado Filho, 2501, Bairro Vila Rio de
Janeiro, Guarulhos.
VI. Campus São João da Boa Vista, situado no Acesso Dr. João Batista Merlin, s/ n.º,
Jardim Itália, São João da Boa Vista.
VII. Campus Caraguatatuba, situado na Avenida Rio Grande do Norte, 450, Indaiá,
Caraguatatuba.
VIII. Campus Bragança Paulista, situado na Avenida Francisco Samuel Lucchesi Filho,
770, Penha, Bragança Paulista.
IX. Campus Salto, situado na Rua Rio Branco, 1780, Vila Teixeira, Salto.
X. Campus São Roque, situado na Rodovia Quintino de Lima, 2100, Jardim Conceição,
São Roque.
XI. Campus São Carlos, situado na Rodovia Washington Luís, km 235 AT-6, Sala 119,
São Carlos.
XII. Campus Campos do Jordão, situado na Rua Monsenhor José Vita, 280, Vila
Abernéssia, Campos do Jordão.
XIII. Campus Barretos, situado na Avenida C-1, Bairro Ide Daher, Barretos.
XIV. Campus Suzano, situado na Avenida Mogi das Cruzes, 1.501, Parque Suzano,
Suzano.
XV. Campus Campinas, situado na Rodovia D. Pedro I – SP 65 – KM 143,6, Amarais,
Campinas;
XVI. Campus Catanduva, situado na Avenida Imperatriz, s/ n.º, Distrito Industrial,
Catanduva.
XVII. Campus Avaré, situado na Avenida Professor Celso Ferreira da Silva, s/ n.º, Jardim
Europa, Avaré.
XVIII. Campus Araraquara, situado no Ramal de Acesso Engenheiro Heitor de Souza
Pinheiro, Araraquara.
XIX. Campus Itapetininga, situado na Avenida João Olímpio de Oliveira, Bairro Assen,
Itapetininga.
XX. Campus Birigui, situado na Rua Pedro Cavalo, 709, Residencial Portal da Pérola II,
Birigui.
XXI. Campus Votuporanga, situado na Rua Pará, s/ n.º, Chácara Aviação, Votuporanga.
XXII. Campus Registro, situado na Rua Clara Gianotti de Souza, 5.180, Agrochá,
Registro.
XXIII. Campus Presidente Epitácio, situado na Rua Rua José Ramos Júnior, nº 27-50,
Jardim Tropical, Presidente Epitácio.
XXIV. Campus Piracicaba, situado na Rodovia Deputado Laércio Corte, s/ n.º, Bairro
Santa Rosa, Piracicaba.
XXV. Campus Hortolândia, situado na Rodovia Municipal TeodorCondiev, 1896,
Hortolândia.
XXVI. Campus Boituva, situado na Avenida Zélia de Lima Rosa, nº 100, Portal dos
Pássaros, Boituva.
XXVII. Campus Capivari, situado na Avenida Ênio Pires de Camargo, 2971, São João
Batista, Capivari.
XXVIII. Campus Matão, situado na Rua José Bonifácio, 1176, Centro, Matão.
XXIX. Campus São José dos Campos, situado na Rodovia Presidente Dutra s/n – km 145,
Jardim Diamante, São José dos Campos.

Referência: Estatuto, Título I, Capítulo I, Art. 1º, §2º
"""
         }
         ,{
            "question": "Quantos campi o IFSP tem? Informe onde você encontrou essa informação",
            "answer": "O IFSP tem 28 campi. Referência: Estatuto, Capítulo I, Art. 1º, §2º"
         }
        ]

#query = "O IFSP tem só ensino médio? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência."
#query = "Quais artigos do Estatuto dizem que o IFSP oferece mestrado e doutorado?"
#query = "O que diz o Art 6º do estatuto do IFSP?"
#query = "O Art 4º do estatuto do IFSP é sobre o que?"

In [80]:
def get_contexts(response):
    contexts = []
    for node in response.source_nodes:
        metadata = ', '.join(node.metadata.values())
        context  = node.get_content()
        contexts.append(f"Fonte: {metadata} # Contextos: {context}")
    return contexts

[
│   'Fonte: Estatuto, TÍTULO I - DA INSTITUIÇÃO, CAPÍTULO I DA NATUREZA E DAS FINALIDADES. # Contextos: Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº. 11.892, de 29 de dezembro de 2008, constitui-se em autarquia federal, vinculada ao Ministério da Educação, detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar. §1º- O IFSP é domiciliado na sede de sua Reitoria, situada na Rua Pedro Vicente, 625, Canindé, São Paulo. §2º- O IFSP é uma instituição de educação superior, básica e profissional, pluricurricular e multicampi, especializada na oferta de educação profissional e tecnológica nas diferentes modalidades de ensino, com base na conjugação de conhecimentos técnicos e tecnológicos com a sua prática pedagógica, nos termos da lei. Tem como sedes, para os fins da legislação educacional: I - Reitoria, no endereço citado no §1º. II - Campus São Paulo, situado na Rua Pedro Vicente, 625, Canindé, São Paulo. III - Campus Cubatão, situado na Rua Maria Cristina, 50, Jardim Casqueiro, Cubatão. IV - Campus Sertãozinho, situado na Rua Américo Ambrósio, 269, Jardim Canaã, Sertãozinho. V - Campus Guarulhos, situado na Avenida Salgado Filho, 2501, Bairro Vila Rio de Janeiro, Guarulhos. VI - Campus São João da Boa Vista, situado no Acesso Dr. João Batista Merlin, s/ n.º, Jardim Itália, São João da Boa Vista. VII - Campus Caraguatatuba, situado na Avenida Rio Grande do Norte, 450, Indaiá, Caraguatatuba. VIII - Campus Bragança Paulista, situado na Avenida Francisco Samuel Lucchesi Filho, 770, Penha, Bragança Paulista. IX - Campus Salto, situado na Rua Rio Branco, 1780, Vila Teixeira, Salto. X - Campus São Roque, situado na Rodovia Quintino de Lima, 2100, Jardim Conceição, São Roque. XI - Campus São Carlos, situado na Rodovia Washington Luís, km 235 AT-6, Sala 119, São Carlos. XII - Campus Campos do Jordão, situado na Rua Monsenhor José Vita, 280, Vila Abernéssia, Campos do Jordão. XIII - Campus Barretos, situado na Avenida C-1, Bairro Ide Daher, Barretos. XIV - Campus Suzano, situado na Avenida Mogi das Cruzes, 1.501, Parque Suzano, Suzano. XV - Campus Campinas, situado na Rodovia D - Pedro I – SP 65 – KM 143,6, Amarais, Campinas; XVI - Campus Catanduva, situado na Avenida Imperatriz, s/ n.º, Distrito Industrial, Catanduva. XVII - Campus Avaré, situado na Avenida Professor Celso Ferreira da Silva, s/ n.º, Jardim Europa, Avaré. XVIII - Campus Araraquara, situado no Ramal de Acesso Engenheiro Heitor de Souza Pinheiro, Araraquara. XIX - Campus Itapetininga, situado na Avenida João Olímpio de Oliveira, Bairro Assen, Itapetininga. XX - Campus Birigui, situado na Rua Pedro Cavalo, 709, Residencial Portal da Pérola II, Birigui. XXI - Campus Votuporanga, situado na Rua Pará, s/ n.º, Chácara Aviação, Votuporanga. XXII - Campus Registro, situado na Rua Clara Gianotti de Souza, 5.180, Agrochá, Registro. XXIII - Campus Presidente Epitácio, situado na Rua Rua José Ramos Júnior, nº 27-50, Jardim Tropical, Presidente Epitácio. XXIV - Campus Piracicaba, situado na Rodovia Deputado Laércio Corte, s/ n.º, Bairro Santa Rosa, Piracicaba. XXV - Campus Hortolândia, situado na Rodovia Municipal TeodorCondiev, 1896, Hortolândia. XXVI - Campus Boituva, situado na Avenida Zélia de Lima Rosa, nº 100, Portal dos Pássaros, Boituva. XXVII - Campus Capivari, situado na Avenida Ênio Pires de Camargo, 2971, São João Batista, Capivari. XXVIII - Campus Matão, situado na Rua José Bonifácio, 1176, Centro, Matão. XXIX - Campus São José dos Campos, situado na Rodovia Presidente Dutra s/n – km 145, Jardim Diamante, São José dos Campos. XXXI - Núcleo Avançado de Assis, situado na Avenida Getúlio Vargas nº 1200, Vila Nova Santana, Assis. § 3º- Para efeito da incidência das disposições que regem a regulação, avaliação e supervisão da Instituição e dos cursos de educação superior, o IFSP é equiparado às universidades federais. § 4º- O IFSP 

In [68]:
faith_evaluator     = FaithfulnessEvaluator()
relevancy_evaluator = RelevancyEvaluator()
correct_evaluator   = CorrectnessEvaluator()
data = []

for pair in tqdm(pairs):
    result       = {}
    query        = pair['question']
    ground_truth = pair['answer']

    result['question']     = query
    result['ground_truth'] = ground_truth

    # Fazendo consulta
    response           = query_engine.query(query)
    result['answer']   = response.response
    result['contexts'] = get_contexts(response)


    # Avaliando Faithfulness
    eval_result                     = faith_evaluator.evaluate_response(response=response)
    result['faithfulness']          = eval_result.score
    result['faithfulness_feedback'] = eval_result.feedback

    # Avaliando Relevancy
    eval_result                  = relevancy_evaluator.evaluate_response(query=query, response=response)
    result['relevancy']          = eval_result.score
    result['relevancy_feedback'] = eval_result.feedback

    # Avaliando Correctness
    eval_result = correct_evaluator.evaluate(
        query=query,
        response=response.response,
        reference=ground_truth,
    )
    result['correctness']          = eval_result.score
    result['correctness_feedback'] = eval_result.feedback

    data.append(result)


df = pd.DataFrame(data)
df

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving with query id None: Qual o nome do IFSP?
Retrieved node with id, entering: node_0
Retrieving with query id node_0: Qual o nome do IFSP?
Retrieved node with id, entering: node_14
Retrieving with query id node_14: Qual o nome do IFSP?
Retrieved node with id, entering: node_2
Retrieving with query id node_2: Qual o nome do IFSP?
Retrieved node with id, entering: node_3
Retrieving with query id node_3: Qual o nome do IFSP?
Retrieved node with id, entering: node_10
Retrieving with query id node_10: Qual o nome do IFSP?


 33%|███▎      | 1/3 [02:00<04:01, 120.82s/it]

Retrieving with query id None: Quais são os campi do IFSP?
Retrieved node with id, entering: node_0
Retrieving with query id node_0: Quais são os campi do IFSP?
Retrieved node with id, entering: node_15
Retrieving with query id node_15: Quais são os campi do IFSP?
Retrieved node with id, entering: node_10
Retrieving with query id node_10: Quais são os campi do IFSP?
Retrieved node with id, entering: node_1
Retrieving with query id node_1: Quais são os campi do IFSP?


 67%|██████▋   | 2/3 [05:01<02:36, 156.12s/it]

Retrieving with query id None: Quantos campi o IFSP tem? Informe onde você encontrou essa informação
Retrieved node with id, entering: node_0
Retrieving with query id node_0: Quantos campi o IFSP tem? Informe onde você encontrou essa informação
Retrieved node with id, entering: node_2
Retrieving with query id node_2: Quantos campi o IFSP tem? Informe onde você encontrou essa informação
Retrieved node with id, entering: node_15
Retrieving with query id node_15: Quantos campi o IFSP tem? Informe onde você encontrou essa informação


100%|██████████| 3/3 [06:30<00:00, 130.30s/it]


,question,ground_truth,answer,faithfulness,relevancy,correctness,correcteness_feedback
0,Qual o nome do IFSP?,"Instituto Federal de Educação, Ciência e Tecno...",The refined answer remains the same:\n\nO nome...,1.0,1.0,5.0,The generated answer is fully correct and rele...
1,Quais são os campi do IFSP?,"II. Campus São Paulo, situado na Rua Pedro Vic...","Os campi do IFSP são:\n\nI - Campus São Paulo,...",0.0,1.0,4.5,The generated answer is almost identical to th...
2,Quantos campi o IFSP tem? Informe onde você en...,"O IFSP tem 28 campi. Referência: Estatuto, Cap...","O IFSP tem 30 campi.\n\nFonte: Documento ""Esta...",0.0,0.0,3.5,The generated answer is relevant to the user q...


In [ ]:
all_no